In [ ]:
import os
import subprocess



In [ ]:
# Function to run a shell command and print its output
def run_command(command):
    print(f"Running command: {command}")
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    print(f"Output:\n{stdout.decode('utf-8')}")
    if stderr:
        print(f"Error:\n{stderr.decode('utf-8')}")


In [ ]:

# Step 1: Clone the repository
# replace ~/Desktop by the path where you want to run this tutorial
run_command("cd ~/Desktop && git clone https://github.com/echarvel3/skimming_scripts-echarvel.git")


In [ ]:

# Step 2: Run the installation script
install_command = """
cd ~/Desktop/skimming_scripts-echarvel
bash ./install.sh << EOF
workshop_test
EOF
"""
run_command(install_command)

In [ ]:
# Step 3: Run the BBMAP commands
bbmap_commands = [
    "cd ~/Desktop/skimming_scripts-echarvel",
    "source activate workshop_test",  # Using the created conda environment
    "cd ./test/",
    "gunzip ./skims/*gz",
    "mkdir -p bbmap_reads",
    "bash ../bbmap_pipeline.sh ./skims/read1.fq ./skims/read2.fq ./bbmap_reads/read_out.fq"
]

# Run the BBMAP commands
for command in bbmap_commands:
    run_command(command)

In [ ]:
# Step 4: Run the Skmer Reference
skmer_commands = [
    "cd ~/Desktop/skimming_scripts-echarvel",
    "source activate workshop_test",  # Using the created conda environment
    "gunzip ./test/skims/*gz",
    "skmer reference ./skims/bbmap_reads -p1"
]

# Run the Skmer Reference commands
for command in skmer_commands:
    run_command(command)

In [ ]:
# Step 5: Run the KRANK build
krank_commands = [
    "cd ~/Desktop/skimming_scripts-echarvel/test/KRANK_test",
    "source activate workshop_test",  # Using the created conda environment
    """krank build -l ${LIBDIR} -t ./taxonomy/ -i ./input_map.tsv -k 27 -w 35 -h 12 -b 8 -s 2 --from-scratch --input-sequences --kmer-ranking representative --adaptive-size --lca soft --num-threads ${NTHREADS}""",
    """krank build -l ${LIBDIR} -t ./taxonomy/ -i ./input_map.tsv -k 27 -w 35 -h 12 -b 8 -s 2 --target-batch 0 --fast-mode --from-library --input-sequences --keep-intermediate --kmer-ranking representative --adaptive-size --lca soft --num-threads ${NTHREADS}"""
]

# Set environment variables for LIBDIR and NTHREADS
os.environ['LIBDIR'] = '/path/to/libdir'  # Replace with actual LIBDIR path
os.environ['NTHREADS'] = '4'  # Replace with the number of threads you want to use

# Run the KRANK build commands
for command in krank_commands:
    run_command(command)

In [ ]:
# Step 6: Run the KRANK query
krank_query_commands = [
    "cd ~/Desktop/skimming_scripts-echarvel/test",
    "source activate workshop_test",  # Using the created conda environment
    """${SCRIPT_DIR}/KRANK/krank query --library-dir ${LIBRARIES} --query-file ./bbmap_reads/read_out.fq --max-match-distance 5 --total-vote-threshold 0.03 --num-threads ${NUM_THREADS} --output-dir "${OUTPUT_DIRECTORY}/krank_output/krank_reports/\""""
]

# Set environment variables for SCRIPT_DIR, LIBRARIES, NUM_THREADS, and OUTPUT_DIRECTORY
os.environ['SCRIPT_DIR'] = '/path/to/scripts'  # Replace with actual SCRIPT_DIR path
os.environ['LIBRARIES'] = '/path/to/libraries'  # Replace with actual LIBRARIES path
os.environ['NUM_THREADS'] = '4'  # Replace with the number of threads you want to use
os.environ['OUTPUT_DIRECTORY'] = '/path/to/output'  # Replace with actual OUTPUT_DIRECTORY path

# Run the KRANK query commands
for command in krank_query_commands:
    run_command(command)

In [ ]:
# Step 7: Run RESPECT
respect_commands = [
    "cd ~/Desktop/skimming_scripts-echarvel/test",
    "source activate workshop_test",  # Using the created conda environment
    "respect -d ./bbmap_reads/ -N 2 --threads 1"
]

# Run the RESPECT commands
for command in respect_commands:
    run_command(command)